In [ ]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
import os
from dotenv import load_dotenv


In [ ]:
load_dotenv()

In [ ]:
# Model configuration
ANTHROPIC_MODEL = os.getenv("ANTHROPIC_MODEL", "claude-sonnet-4-20250514")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4.1")  
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY2")

# Directory configuration
TEMP_DIR = "temp"

In [ ]:
CATEGORIZER_SYSTEM_MESSAGE = """
You are an AI financial analyst. Your purpose is to categorize financial transactions into a few broad categories.

You will receive a JSON object that contains:
1. 'transactions_by_cardholder': a dictionary where each key is a cardholder name and    the value is a list of transaction objects.
2. 'summary': a dictionary with account summary data.

Your job:
- Return the exact same JSON object structure.
- Do NOT remove or rename any keys.
- Do NOT modify the 'summary' section.
- Inside 'transactions_by_cardholder', for each transaction object, add a new key-value   pair: "category": "Category Name".

CRITICAL RULES:
Use ONLY the 6 categories defined below.
For payments, refunds, and fees, use the Financial Transactions category.
If a description is too vague, use Uncategorized.

CATEGORY DEFINITIONS:
Food & Dining: All food-related spending. This includes both groceries from supermarkets and purchases from restaurants, cafes, bars, and food delivery services.
Merchandise & Services: A broad category for general shopping and personal care. This includes retail stores, online marketplaces (like Amazon), electronics, clothing, hobbies, entertainment, streaming services (Netflix), gym memberships, and drugstores (CVS).
Bills & Subscriptions: Recurring charges for essential services. This primarily includes utilities (phone, internet) and insurance payments.
Travel & Transportation: Costs for getting around. This includes daily transport (gas stations, Uber, public transit) and long-distance travel (airlines, hotels, rental cars).
Financial Transactions: All non-spending activities that affect your balance. This includes payments made to your account, refunds from merchants, statement credits, and any fees or interest charges.
Uncategorized: For any transaction that does not clearly fit into the categories above.

Output ONLY the JSON with categories added. Do not include any explanations or markdown formatting. Once you output the categorized JSON, you are done - do not continue the conversation.
"""

In [ ]:
def get_openai_client():
    """Get configured OpenAI model client."""
    return OpenAIChatCompletionClient(
        model=OPENAI_MODEL, 
        api_key=OPENAI_API_KEY
    )


In [ ]:
model_client = get_openai_client()

# Code executor
code_executor = DockerCommandLineCodeExecutor(
    work_dir=TEMP_DIR,
    image="amancevice/pandas",  
    )
await code_executor.start()

In [ ]:
# STAGE 2: Categorizer processes the parsed JSON
# --- Categorizer Agent ---
categorizer_agent = AssistantAgent(
    name="categorizer",
    model_client=model_client,
    system_message=CATEGORIZER_SYSTEM_MESSAGE,
    reflect_on_tool_use=False
        )

categorizer_task = TextMessage(
    content=f"Here is the parsed JSON to categorize:\n```json\n{json.dumps(parsed_json, indent=2)}\n```",
    source="user"
)

max_message_termination = MaxMessageTermination(max_messages=5)

categorizer_team = RoundRobinGroupChat(
    participants=[categorizer_agent],
    termination_condition=max_message_termination
)

categorization_result = await Console(categorizer_team.run_stream(task=categorizer_task))

# Stop executor safely
await code_executor.stop()

# Search categorization result for final JSON
final_parsed_json = None

# Check categorizer messages first
for msg in categorization_result.messages:
    src = getattr(msg, "source", "")
    content = getattr(msg, "content", None)
    try:
        content_str = content if isinstance(content, str) else str(content)
    except Exception:
        content_str = None

    if not content_str:
        continue

    if src == "categorizer":
        final_parsed_json = extract_json_from_text(content_str)
        if final_parsed_json is not None:
            break